In [1]:
#MIT License
#
#Copyright (c) 2020 Jaime Meléndez / Carlos Tellería
#
#Permission is hereby granted, free of charge, to any person obtaining a copy
#of this software and associated documentation files (the "Software"), to deal
#in the Software without restriction, including without limitation the rights
#to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
#copies of the Software, and to permit persons to whom the Software is
#furnished to do so, subject to the following conditions:
#
#The above copyright notice and this permission notice shall be included in all
#copies or substantial portions of the Software.
#
#THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
#IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
#FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
#AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
#LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
#OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
#SOFTWARE.
#

In [2]:
import itertools
#import prmskmeans as km
#from pandarallel import pandarallel
import pandas as pd
import random
#import matplotlib.pyplot as plt
#import matplotlib.cm as cm
import time
#import plotly.express as px
import numpy as np
from multiprocessing import cpu_count, Pool
import math

pathD = "def/clust"


In [3]:
def l2_distance(point1, point2):
    return math.sqrt(sum((float(i)-float(j))**2 for (i,j) in zip(point1, point2)))

def parallelize(data, func):
    cores = cpu_count()
    print("processing", cores, "cores")
    data_split = np.array_split(data, cores)
    pool = Pool(cores)
    data = pd.concat(pool.map(func, data_split))
    pool.close()
    pool.join()
    return data



def cluster_quality(cluster):
    cluster_quality.counter += 1
    print(cluster_quality.counter)
    if len(cluster) == 0:
        return 0.0
    quality = 0.0

    #t = set(range(len(cluster)))
    #comb = pd.DataFrame(set(itertools.combinations(t,2)))
    for i in range(len(cluster)):
            for j in range(i, len(cluster)):
                quality += l2_distance(cluster[i], cluster[j])
    """
    dist = comb.apply(lambda row: l2_distance(cluster[row[0]], cluster[row[1]]), axis=1)
    """
    return quality / len(cluster)

cluster_quality.counter=0


def par_cluster_quality(cllist):
    return cllist.map(lambda row: cluster_quality(row))


def compute_quality(data, cluster_indices):
    clusters = dict()
    for i, c in enumerate(cluster_indices):
        if c in clusters:
            clusters[c].append(data.iloc[i][['x','y']])
        else:
            clusters[c] = [data.iloc[i][['x','y']].values]
    print(clusters)
    return sum(cluster_quality(c) for c in clusters.values())




In [4]:
#Distancia entre todos los puntos dividido entre el número de elementos del dataset
#codpos --> provincia, Aragón
tic = time.time()
readF = pathD + "_rkm.csv"
df = pd.read_csv(readF)
data_filtered = df[['x','y','cluster']].dropna()
data_filtered['point'] = list(zip(data_filtered.x, data_filtered.y))
final = data_filtered.drop('x',axis=1).drop('y',axis=1)
#clusters = data_filtered['cluster'].tolist()
dt = final.sort_values('cluster')
init = dt.groupby('cluster')['point'].apply(list)
#sums = init.parallel_apply(lambda row: cluster_quality(row))
sums = parallelize(init, par_cluster_quality)
toc = time.time()

print("excution_time: ", toc-tic)
intra = sums.sum()/len(data_filtered)
print("Calidad intra cluster: ", intra)

processing 4 cores
1
1
1
1
2
2
2
3
3
2
3
3
4
4
5
6
5
4
5
6
6
7
4
8
5
7
6
7
8
7
8
8
9
10
11
12
13
9
14
9
10
911

10
12
15
10
11
11
16
12
13
17
12
13
13
18
14
15
16
17
18
14
14
15
19
15
16
19
20
21
20
16
22
23
17
18
19
17
24
18
25
26
19
20
21
22
23
21
20
24
22
23
21
22
23
24
25
24
26
25
25
26
excution_time:  58.21265530586243
Calidad intra cluster:  1838.7101375991917


In [5]:
tic = time.time()
readF = pathD + "_centers.csv"
df = pd.read_csv(readF)
data_filtered = df[['x','y']].dropna()
data_filtered['point'] = list(zip(data_filtered.x, data_filtered.y))
final = data_filtered.drop('x',axis=1).drop('y',axis=1)
#clusters = data_filtered['cluster'].tolist()
"""
dt = final.sort_values('cluster')
init = dt.groupby('cluster')['point'].apply(list)
#sums = init.parallel_apply(lambda row: cluster_quality(row))
sums = parallelize(init, par_cluster_quality)
"""

toc = time.time()

print("excution_time: ", toc-tic)
#sums.sum()/len(data_filtered)

sums = cluster_quality(final['point'].values.tolist())

inter = sums/len(data_filtered)
print("Calidad inter cluster: ", inter)

excution_time:  0.0525362491607666
1
Calidad inter cluster:  28251.45717188631


In [6]:
print("Quality: ", inter/intra)

Quality:  15.364823739305809
